<a href="https://colab.research.google.com/github/PTson2207/Deployment-Model-App/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training someone model, then deploy ...

- Want: Save_model and then upload Google Storage
- Data: From Kaggle-Food 101
- Model(s): 10, 11, 12 classes

Setup some Function

In [1]:
# library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [2]:
# Function unzip dowload_data file
def unzip_data(file_name):
    zip_ref = zipfile.ZipFile(file_name, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [3]:
# Setup dataset input
IMG_SIZE = (224, 224)

def create_data_loader(train_dir, test_dir, img_size=IMG_SIZE):
    train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                     label_mode="categorical",
                                                                     img_size=img_size)
    
    test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                    label_mode="categorical",
                                                                    img_size=img_size)
    return train_data, test_data

In [5]:
data_augument = keras.Sequential([
                                  preprocessing.RandomFlip('horizontal'),
                                  preprocessing.RandomRotation(0.2),
                                  preprocessing.RandomHeight(0.2),
                                  preprocessing.RandomWidth(0.2),
                                  preprocessing.RandomZoom(0.2),
                                  #preprocessing.Rescaling(1./255) # for Resnet50V2, removing for EfficientNetB0
], name="data_augument")

In [7]:
# setup input_shape and base_model
INPUT_SHAPE = (244, 244, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

In [8]:
def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
    # Fine tune model
    base_model.trainable = False
    # create layers
    inputs = layers.Input(shape=INPUT_SHAPE, name="input_layer")
    # add data augument sequentital as a layer
    x = data_augument(inputs)
    # give model inputs (after augument) but don't train it
    x = base_model(x, training= False)
    # pool out_put fearture os base_model
    x = layers.GlobalAveragePooling2D(name='pooling_layers')(x)
    # put a dense layer as a out_put
    outputs = layers.Dense(num_classes, activation='softmax', name='output_layers')(x)

    # make a model from input and output
    model = keras.Model(inputs, outputs)

    model.complie(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    model.summary()

    return model